# Pinyin Syllable Count Generator Working Draft Notes
This document details the process of experimentation related to the construction of the Pinyin Syllable Count Generator function. The function takes pinyin of any size or word count, evaluates the number of syllables based on the supplied Latin characters, and outputs a list of integers representing the syllables of each word.

The initial stages of development tests three methods of storing data for optimized processing comparison: Python list, NumPy Array, and Pandas DataFrame.

In [2]:
import pandas as pd
import numpy as np

Because pinyin is a structured system with accepted combinations of "initials" (the initial sound of a syllable, e.g., the first consonant) and "finals" (the ending sound of a syllable, e.g., vowels that sometimes end with a consonant). One syllable in pinyin can be as small as one letter and as large as six letters. Therefore, detecting a syllable can occur in one of two ways:
1. A sequence of pinyin letters compared to a list of accepted pinyin syllables. If the sequence appears in the list, or if the sequence is part of a longer accepted term ("tia" by itself is not an accepted syllable, however "tiao" and "tian" are), then an additional letter from the evaluated pinyin can be added to the sequence and reevaluated. If the sequence does not appear in the list and is not part of a longer accepted term, then the last character of the sequence becomes the first character of the next syllable, and the syllable count incremented.
2. A sequence of letters is processed incrementally with the initials identified first, and then the finals after the appearance of the first vowel. The identified initial and final values act as indexes to a table (either a NumPy Array or Pandas DataFrame) of boolean values that evaluate the initial and final pairing. If the table returns False, then the final is compared to a list of other final values to determine whether it can be part of a more extended final. If it is, then the function continues onto the next incremented letter. Otherwise, the function determines that the current letter is the start of the next syllable and increments the number of syllables for that word.

Given these possibilities, two metrics need to be considered to settle on the proper method for the evaluation function: speed and size. I hypothesize that comparing a sequence of letters to a list of terms can be faster; however, the list itself may be excessively large. Evaluating sequences by their combination of initials and finals, however, may take longer but results in a much smaller package. The following exercises test each method on a smaller scale to get a general sense of the speed vs. size issue.

## Setup for Testing

The code below imports two CSV files. The first contains the list of acceptable pinyin syllables (e.g. "yang," "wei"). The second contains an array of 1 values with a column header of initials and row headers with finals. Empty values convert to 0, and both 1 and 0 values convert to the boolean values True and False, respectively in their array and DataFrame forms.

In [5]:
import csv
import sys

## LIST ##
# Import list of all pinyin combinations
smList = []
with open("pinyinList.csv") as f:
    r = csv.reader(f)
    for row in r:
        smList += row
        
## NUMPY/DATAFRAME ##
# List of initials, finals for Numpy evaluation
initList = list(np.genfromtxt('pinyinDF.csv', delimiter=',',dtype=str, usecols=0)[1:])
finList = list(np.genfromtxt('pinyinDF.csv', delimiter=',',dtype=str)[0][1:])

# Import Numpy array from CSV
smAr = np.genfromtxt('pinyinDF.csv', delimiter=',', skip_header=1, filling_values=0
                    ).astype('bool_')
smAr = np.delete(smAr, 0, 1)

# Import dataframe from CSV 
smDF = pd.read_csv('pinyinDF.csv', header=0, index_col=0, dtype={'INDEX':str})
smDF = smDF.fillna(0).astype('bool')

# Print examples of each form of reference data.
print('Python List:')
print(str(smList[0:30]))
print('Displaying 30 of ' + str(len(smList)) + ' possible pinyin combinations ' + '\n')
print('——————————————————\n')
print('# Header lists for use in indexing the array/DataFrame')
print('Initials List:')
print(initList)
print('Finals List:')
print(str(finList) + '\n')
print('——————————————————\n')
print('Numpy Array:')
print(str(smAr[0]))
print('Displaying 1 of ' + str(len(smAr)) + ' rows within the array' + '\n')
print('Pandas DataFrame:')
print(smDF.head(3))

Python List:
['a', 'ai', 'ao', 'an', 'ang', 'e', 'ei', 'en', 'eng', 'er', 'o', 'ou', 'yi', 'ya', 'yao', 'ye', 'you', 'yan', 'yang', 'yin', 'ying', 'yong', 'wu', 'wa', 'wo', 'wei', 'wai', 'wan', 'wen', 'wang']
Displaying 30 of 407 possible pinyin combinations 

——————————————————

# Header lists for use in indexing the array/DataFrame
Initials List:
['ø', 'b', 'p', 'm', 'f', 'd', 't', 'n', 'l', 'z', 'c', 's', 'zh', 'ch', 'sh', 'r', 'j', 'q', 'x', 'g', 'k', 'h', 'y', 'w']
Finals List:
['a', 'ai', 'ao', 'an', 'ang', 'e', 'ei', 'en', 'eng', 'er', 'o', 'ong', 'ou', 'i', 'ia', 'iao', 'ie', 'iu', 'ian', 'iang', 'in', 'ing', 'iong', 'u', 'ua', 'uo', 'ue', 'ui', 'uai', 'uan', 'uang', 'un', 'v', 've']

——————————————————

Numpy Array:
[ True  True  True  True  True  True  True  True  True  True  True False
  True False False False False False False False False False False False
 False False False False False False False False False False]
Displaying 1 of 24 rows within the array

Pandas DataFram

## Performing the Tests
To best compare the size and performance speed of each method this test works with two examples: a small two-letter phrase that is an acceptable pinyin term ("ba"), and a longer five-letter term that is not an acceptable pinyin term ("quang"). The %timeit function measures processing time with each method after running 100,000 iterations.

### Methods
1. The Python "in" function: the script searches through the list of acceptable pinyin terms for the supplied example.
2. The examples split into their initial and final pairs, their indexes derived from within the list of initials and finals, and those values are then used to find the appropriate boolean within the NumPy Array.
3. Pandas DataFrame 

In [10]:
# Variables used for testing
smEval1 = 'ba'
smInitial1 = 'b'
smFinal1 = 'a'

smEval2 = 'quang'
smInitial2 = 'q'
smFinal2 = 'uang'


# Subject 1: Short, acceptable term
print('# Results of \"ba\" (short, valid):')

# Method 1: Python List
print('Python List: ' + str(sys.getsizeof(smList)))
%timeit -n 100000 smEval1 in smList

# Method 2: Numpy Array
print('\nNumpy array: ' + str(sys.getsizeof(initList) +
                              sys.getsizeof(finList) +
                              sys.getsizeof(smAr)) +
      ', without initials and finals lists: ' + str(sys.getsizeof(smAr)))
%timeit -n 100000 smAr[initList.index(smInitial1),finList.index(smFinal1)] != True

# Method 3: Pandas DataFrame
# 3a: slice
print('\nDataFrame: ' + str(sys.getsizeof(smDF)))
print('slice')
%timeit -n 100000 smDF[smFinal1][smInitial1] != True

# 3b: loc
print('loc')
%timeit -n 100000 smDF.loc[smInitial1,smFinal1] != True

print('iloc')
# 3c: iloc
%timeit -n 100000 smDF.iloc[initList.index(smInitial1),finList.index(smFinal1)] != True



# Subject 2: Longer, unacceptable term
print('\n\n# Results of \"quang\" (longer, invalid):')

# Method 1: Python List
print('Python List: ' + str(sys.getsizeof(smList)))
%timeit -n 100000 smEval2 in smList

# Method 2: Numpy Array
print('\nNumpy array: ' + str(sys.getsizeof(initList) +
                              sys.getsizeof(finList) +
                              sys.getsizeof(smAr)) +
      ', without initials and finals lists: ' + str(sys.getsizeof(smAr)))
%timeit -n 100000 smAr[initList.index(smInitial2),finList.index(smFinal2)] != True

# Method 3: Pandas DataFrame
# 3a: slice
print('\nDataFrame: ' + str(sys.getsizeof(smDF)))
print('slice')
%timeit -n 100000 smDF[smFinal2][smInitial2] != True

# 3b: loc
print('loc')
%timeit -n 100000 smDF.loc[smInitial2,smFinal2] != True

print('iloc')
# 3c: iloc
%timeit -n 100000 smDF.iloc[initList.index(smInitial2),finList.index(smFinal2)] != True

# Results of "ba" (short, valid):
Python List: 3760
425 ns ± 15.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Numpy array: 1672, without initials and finals lists: 928
1.44 µs ± 49 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

DataFrame: 3067
slice
9.98 µs ± 327 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
loc
7.93 µs ± 78.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
iloc
9.11 µs ± 260 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Results of "quang" (longer, invalid):
Python List: 3760
3.94 µs ± 32.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Numpy array: 1672, without initials and finals lists: 928
1.9 µs ± 48.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

DataFrame: 3067
slice
9.38 µs ± 168 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
loc
7.58 µs ± 73.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
iloc
9.43 µs ± 146 ns per loop (mean ± std.

In [61]:
import numpy as np
import csv

# List of initials, finals for Numpy evaluation
initList = list(np.genfromtxt('pinyinDF.csv', delimiter=',',dtype=str, usecols=0)[1:])
finList = list(np.genfromtxt('pinyinDF.csv', delimiter=',',dtype=str)[0][1:])

# Import Numpy array from CSV
smAr = np.genfromtxt('pinyinDF.csv', delimiter=',', skip_header=1, filling_values=0
                    ).astype('bool_')
smAr = np.delete(smAr, 0, 1)

def sylCount(pinyin, test=False, errorReport=False):
    '''Count the number of Chinese characters based on inputted pinyin text.'''
    
    # Lists for reference
    vowel = ['a','e','i','o','u','v']
    
    # Variables used for pinyin evaluation
    # Replace ideographic spaces with regular space, also trim leading and following spaces.
    pinyin = pinyin.replace('\u3000',' ').strip()
    
    # Make sure no symbols get through. Throw out null value if so.
    if not pinyin.replace(' ','').replace('\'','').isalpha():
        errorMsg = 'Error: invalid non-alpha characters in name'
        if test == True: print(errorMsg)
        if errorReport == True: return errorMsg
        return [0]
    
    # CBDB testing purposes ONLY: if the family name is duplicated in the given name
    dupTest = pinyin.split()
    if len(dupTest) > 1:
        dupTestLen = [len(dupTest[0]), len(dupTest[1])]
        if (dupTest[0] == dupTest[1][0:dupTestLen[0]] and 
            dupTestLen[1] > dupTestLen[0] and 
            dupTest[1][len(dupTest[0])] != '\''):
            errorMsg = 'Error: duplicate family name in given name'
            if test == True: print(errorMsg)
            if errorReport == True: return errorMsg 
            return [0]

    initial = ''
    final = ''
    newSyl = False
    syl = 1
    sylCount = []
    exFin = False
    
    if test == True: print('Testing ' + pinyin)
    
    for i,n in enumerate(str.lower(pinyin)):

        if n == '\'':
            step = '0. new syl'
            notes = 'apostrophe'
            newSyl = True
        
        # If current character is a space...
        elif n == ' ':
            step = '0. new word'
            if pinyin[i+1] == ' ':
                notes = 'discard extra space'
            else:
                sylCount.append(syl)
                newSyl = True
                notes = 'space'

        # If final has already been set, regardless of current character
        elif final:
            step = 'b. ' + initial + '|' + final + n
            if (final[-1] == 'e' and n == 'r') or (final[-1] == 'n' and n == 'g'):
                notes = final[-1] + n
                if i != len(pinyin) - 1 and pinyin[i+1] != ' ':
                    newSyl = True
                    exFin = final[-1] + n
                    notes += ' - not \"' + exFin + '\" final'
                else:
                    if notes == 'er' and len(final) > 1:
                        syl += 1
                        notes += ' - extra syl at end'

            else:
                notes = 'fin'
                try:
                    initIdx = initList.index(initial)
                    finIdx = finList.index(final + n)
                    result = smAr[initIdx,finIdx]
                    notes += ' - trying ' + initial + final + n + ': ' + str(result)
                except:
                    result = False
                    notes += ' - array error'

                if not result:
                    listFins = list(i for i in finList if i.startswith(final+n))
                    if not listFins or not any(i for i in listFins if smAr[initList.index(initial),finList.index(i)]):
                        newSyl = True
                        notes += ' - new syl'
                    else:
                        final += n
                        notes += ' - definitely more'
                else:
                    final += n
                    notes += ' - maybe more'
          
        # If there is no final set (work on initial)
        else:
            step = 'a. ' + initial + n
            notes = 'init'
            if n not in vowel:
                if initial + n not in initList:
                    errorMsg = 'Error: ' + initial + n + ' - quitting, invalid initials'
                    if test == True: print(errorMsg)
                    if errorReport == True: return errorMsg
                    return [0]
                initial += n
                notes += ' - cons'
            elif not initial:
                initial = 'ø'
                final = n
                notes += ' - starting vowel'
            else:
                final = n
                notes += ' - vowel'
                
        # Print debug notes if parameter is true
        if test == True: print(step + ': ' + notes)
        
        # Processes for next syllable
        if newSyl:
            if n in vowel:
                initial = 'ø'
                final = n
                syl += 1
            elif n != ' ':
                if (exFin and pinyin[i+1] not in vowel) or n == '\'':
                    initial = ''
                    if exFin == 'er' and len(final) > 1:
                        syl+= 1
                else:
                    initial = n
                exFin = False
                final = ''
                syl += 1
            else:
                initial = ''
                final = ''
                syl = 1
            if test == True: print('new syl: ' + str(syl))
            newSyl = False
    
    sylCount.append(syl)
    
    if errorReport == True:
        return np.nan
    else:
        return sylCount

In [62]:
import pandas as pd

# Import CBDB data
names = pd.read_csv('pinyinTestNames.csv', names=['EngName','ChiName'], header=0)

# Run sylCount function on 'EngName' column
names['syl'] = names['EngName'].apply(sylCount, errorReport=True)

# Add 'check' column to validate results
def tryAdd(x):
    try:
        return sum(x)
    except:
        return
names['check'] = names['ChiName'].str.len() == names['syl'].apply(tryAdd)

# Filter results to show false negatives
names[~names['check']]

,EngName,ChiName,syl,check
0,A Chou,阿丑,NaN,False
1,A Duoni,阿多尼,NaN,False
2,A Erqin,阿爾親,NaN,False
3,A Hada,阿哈達,NaN,False
4,A Hadan,阿哈旦,NaN,False
5,A Hashan,阿哈善,NaN,False
6,A Honga,阿洪阿,NaN,False
7,A Ji,阿寄,NaN,False
8,A Keshan,阿克山,NaN,False
9,A Lejin,阿勒金,NaN,False


In [63]:
names[names['syl'].notnull()]

,EngName,ChiName,syl,check
109,Ai Airen,愛仁,Error: duplicate family name in given name,False
110,Ai Aixinga,愛興阿,Error: duplicate family name in given name,False
437,Aixinjueluo Fufong,愛新覺羅富宏,"Error: ng - quitting, invalid initials",False
697,Aixinjueluo Qining,愛新覺羅耆寧,"Error: ng - quitting, invalid initials",False
1168,An Anduha,安篤哈,Error: duplicate family name in given name,False
1169,An Anfu,安福,Error: duplicate family name in given name,False
1170,An Anling,安齡,Error: duplicate family name in given name,False
2021,Bai Baichun,柏春,Error: duplicate family name in given name,False
2022,Bai Baidaer,拜達兒,Error: duplicate family name in given name,False
2023,Bai Bailin,白麟,Error: duplicate family name in given name,False


In [45]:
countAccuracy = (round(len(names[~names['check']][names['syl'].apply(sum) > 0][~names['ChiName'].str.contains('\(')]) / len(names) * 100, 2) - 100) * -1
print(str(countAccuracy) + '%')

99.67%


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
